In [55]:
import warnings

import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np

from joblib import load
from numpy import genfromtxt

import pickle
import pandas as pd
import os
from tqdm import tqdm

In [2]:
# Import Model
def import_model():
    list_model = []
    path = './models'
    for i in os.listdir(path):
        list_model.append(i)
    print(list_model)
    return list_model

In [3]:
# Import normal Dataset
def import_dataset():
    with open('./Dataset/csv/Attack_merge.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
    return X_train, X_test, y_train, y_test

In [4]:
import itertools
def combinations(list_model, num):
    return [x for x in itertools.combinations(list_model, num)]

In [5]:
combination_model = combinations(['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib'], 3)
print(type(combination_model[0]))

<class 'tuple'>


In [6]:
def kappa_statistics(pred_labels, y_test):
    ks = 0
    length = len(pred_labels)
    for n in range(len(pred_labels[0])):
        # value_b = ''
        for o in range(0, length):
            if o == 0:
                continue
            elif pred_labels[o][n] != pred_labels[o-1][n]:
                break
            elif o == length-1:
                ks += 1
            else:
                continue
    ks_percentage = ks/len(y_test) * 100
    #print("KAPPA STATS: " + str(ks_percentage))

    return ks_percentage

In [7]:
def double_fault(pred_labels, y_test):
    wrong_gt = ([l for l, m in enumerate(y_test) if m != pred_labels[0][l]])
    df = 0
    length = len(pred_labels)
    for n, o in enumerate(wrong_gt):
        for p in range(0, length):
            if p == 0:
                continue
            elif pred_labels[p][o] != pred_labels[p-1][o]:
                break
            elif p == length-1:
                df += 1
            else:
                continue
    df_percentage = df/len(y_test) * 100
    #print("DOUBLE FAULT: " + str(df_percentage))
    
    return df_percentage

In [50]:
def prediction_label(model, X_test, X_test_ex, mode=0):
    path = str("./models/")
    if model == "CNN.h5" or model == "LSTM.h5":
        clf = tf.keras.models.load_model(path + model)
        prediction = clf.predict_proba(X_test_ex)
        prediction_prob = tf.keras.models.load_model(path + model).predict_proba(X_test_ex).flatten()

    elif model == "DNN4.h5":
        prediction = tf.keras.models.load_model(path + model).predict(X_test)
        prediction_prob = tf.keras.models.load_model(path + model).predict_proba(X_test)[0:,1]
        prediction = prediction[0:,1]

    elif model == "SVM.joblib":
        prediction = load(path + model).predict(X_test)
        prediction_prob = load(path + model)._predict_proba_lr(X_test)[0:,1]
        print("prediction_prob",prediction_prob)
        
    else:
        prediction = load(path + model).predict(X_test)
        prediction_prob = load(path + model).predict_proba(X_test)[0:,1]

    if mode == 0:
        return np.rint(prediction)
    else:
        return prediction_prob

In [52]:
def voting(good_team, X_test, X_test_ex, y_test):
    #soft voting
    f1_score =[]
    for i in range(len(good_team)):
        #print(good_team[i])
        pred = np.array([prediction_label(model, X_test, X_test_ex, mode=1) for model in good_team[i]])
        pred = np.average(pred, axis=0, weights=None)
        pred_labels = np.rint(pred)
        f1 = sklearn.metrics.f1_score(y_test, pred_labels, average="weighted")
        f1_score.append(f1)
        #print("f1_score",f1_score)

    print(good_team)
    print(f1_score)
    f1_score_max = np.max(f1_score)
    best_team_index = np.argmax(f1_score)

    return good_team[best_team_index], f1_score_max

In [51]:
X_train, X_test, y_train, y_test = import_dataset()
y_train_ex = tf.expand_dims(y_train, axis=1)
X_train_ex = tf.expand_dims(X_train, axis=2)
y_test_ex = tf.expand_dims(y_test, axis=1)
X_test_ex = tf.expand_dims(X_test, axis=2)
good_team = [('CNN.h5', 'SVM.joblib'), ('KNN.joblib', 'LR.joblib'), ('CNN.h5', 'SVM.joblib'), ('LR.joblib', 'XGB.joblib'), ('LSTM.h5', 'SVM.joblib'), ('SVM.joblib', 'XGB.joblib')]
best_team, best_team_f1 = voting(good_team, X_test, X_test_ex, y_test)
print("best_team, best_team_f1", best_team, best_team_f1)


('CNN.h5', 'SVM.joblib')


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [2.7592180e-20 4.4376504e-01 1.0000000e+00 ... 9.9962938e-01 8.3827194e-20
 2.0672667e-14]
prediction_prob [0.14430111 0.47238292 0.73114852 ... 0.63567313 0.15379648 0.30297049]
f1_score [0.9293144577144177]
('KNN.joblib', 'LR.joblib')
f1_score [0.9293144577144177, 0.9739940855483459]
('CNN.h5', 'SVM.joblib')


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [2.7592180e-20 4.4376504e-01 1.0000000e+00 ... 9.9962938e-01 8.3827194e-20
 2.0672667e-14]
prediction_prob [0.14430111 0.47238292 0.73114852 ... 0.63567313 0.15379648 0.30297049]
f1_score [0.9293144577144177, 0.9739940855483459, 0.9293144577144177]
('LR.joblib', 'XGB.joblib')
f1_score [0.9293144577144177, 0.9739940855483459, 0.9293144577144177, 0.975353067271378]
('LSTM.h5', 'SVM.joblib')


e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [8.9635706e-04 1.0109200e-01 9.9833703e-01 ... 9.9833858e-01 8.8764552e-04
 8.8759640e-04]
prediction_prob [0.14430111 0.47238292 0.73114852 ... 0.63567313 0.15379648 0.30297049]
f1_score [0.9293144577144177, 0.9739940855483459, 0.9293144577144177, 0.975353067271378, 0.9695934546816005]
('SVM.joblib', 'XGB.joblib')
prediction_prob [0.14430111 0.47238292 0.73114852 ... 0.63567313 0.15379648 0.30297049]
f1_score [0.9293144577144177, 0.9739940855483459, 0.9293144577144177, 0.975353067271378, 0.9695934546816005, 0.9808880568881341]
[('CNN.h5', 'SVM.joblib'), ('KNN.joblib', 'LR.joblib'), ('CNN.h5', 'SVM.joblib'), ('LR.joblib', 'XGB.joblib'), ('LSTM.h5', 'SVM.joblib'), ('SVM.joblib', 'XGB.joblib')]
[0.9293144577144177, 0.9739940855483459, 0.9293144577144177, 0.975353067271378, 0.9695934546816005, 0.9808880568881341]
best_team, best_team_f1 ('SVM.joblib', 'XGB.joblib') 0.9808880568881341


e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [56]:
def ensemble_learning(num_teams):
    good_team = []
    f1_score_team = []
    list_model = import_model()
    X_train, X_test, y_train, y_test = import_dataset()
    y_train_ex = tf.expand_dims(y_train, axis=1)
    X_train_ex = tf.expand_dims(X_train, axis=2)
    y_test_ex = tf.expand_dims(y_test, axis=1)
    X_test_ex = tf.expand_dims(X_test, axis=2)

    combination_model = combinations(list_model=list_model, num = num_teams)
    # print(combination_model)
    for j in tqdm(range(0, len(combination_model))):
        pred_labels = []
        for k in range(0, len(combination_model[j])):
            pred_labels.append(prediction_label(combination_model[j][k], X_test, X_test_ex, mode=0))
            
        #print(pred_labels)
        # print(combination_model[j])
        # print(f1_inteam)
        ks_percentage = kappa_statistics(pred_labels, y_test)
        df_percentage = double_fault(pred_labels, y_test)
        
        if ks_percentage > 50 and ks_percentage < 91 and df_percentage < 50:
            good_team.append(combination_model[j])
            
    print("good_team",good_team)
    best_team, best_team_f1 = voting(good_team, X_test, X_test_ex, y_test)
    print("best_team, best_team_f1", best_team, best_team_f1)
    



In [48]:
ensemble_learning(2)

['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib']


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [[8.8283563e-01]
 [1.0000000e+00]
 [8.3827511e-20]
 ...
 [8.3827194e-20]
 [6.6263997e-06]
 [9.9962938e-01]]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [[8.8283563e-01]
 [1.0000000e+00]
 [8.3827511e-20]
 ...
 [8.3827194e-20]
 [6.6263997e-06]
 [9.9962938e-01]]


...

prediction_prob [[8.8283563e-01]
 [1.0000000e+00]
 [8.3827511e-20]
 ...
 [8.3827194e-20]
 [6.6263997e-06]
 [9.9962938e-01]]
prediction_prob [0.73092541 0.73113926 0.15379634 ... 0.15379655 0.35862518 0.63572931]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [[8.8283563e-01]
 [1.0000000e+00]
 [8.3827511e-20]
 ...
 [8.3827194e-20]
 [6.6263997e-06]
 [9.9962938e-01]]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` i

KeyboardInterrupt: 

In [53]:
ensemble_learning(3)

['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib']


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [0.4437651  0.8828356  0.9996294  ... 0.44376504 1.         0.9996294 ]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [0.4437651  0.8828356  0.9996294  ... 0.44376504 1.         0.9996294 ]


...

e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [0.10305118 0.998334   0.99833846 ... 0.10006356 0.99833703 0.9983386 ]
prediction_prob [0.47237927 0.73100473 0.63557228 ... 0.47238484 0.73113824 0.63567946]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


prediction_prob [0.47237927 0.73100473 0.63557228 ... 0.47238484 0.73113824 0.63567946]
[('CNN.h5', 'DT.joblib', 'LR.joblib'), ('CNN.h5', 'DT.joblib', 'SVM.joblib'), ('CNN.h5', 'KNN.joblib', 'LR.joblib'), ('CNN.h5', 'KNN.joblib', 'SVM.joblib'), ('CNN.h5', 'LR.joblib', 'LSTM.h5'), ('CNN.h5', 'LR.joblib', 'XGB.joblib'), ('CNN.h5', 'LSTM.h5', 'SVM.joblib'), ('CNN.h5', 'SVM.joblib', 'XGB.joblib'), ('DNN4.h5', 'DT.joblib', 'LR.joblib'), ('DNN4.h5', 'DT.joblib', 'LSTM.h5'), ('DNN4.h5', 'DT.joblib', 'SVM.joblib'), ('DNN4.h5', 'KNN.joblib', 'LR.joblib'), ('DNN4.h5', 'KNN.joblib', 'LSTM.h5'), ('DNN4.h5', 'KNN.joblib', 'SVM.joblib'), ('DNN4.h5', 'KNN.joblib', 'XGB.joblib'), ('DNN4.h5', 'LR.joblib', 'LSTM.h5'), ('DNN4.h5', 'LR.joblib', 'XGB.joblib'), ('DNN4.h5', 'LSTM.h5', 'SVM.joblib'), ('DNN4.h5', 'LSTM.h5', 'XGB.joblib'), ('DNN4.h5', 'SVM.joblib', 'XGB.joblib'), ('DT.joblib', 'KNN.joblib', 'LR.joblib'), ('DT.joblib', 'KNN.joblib', 'SVM.joblib'), ('DT.joblib', 'LR.joblib', 'LSTM.h5'), ('DT.jobl

e:\anaconda\envs\Adversarial_attack\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [57]:
ensemble_learning(5)

['CNN.h5', 'DNN4.h5', 'DT.joblib', 'KNN.joblib', 'LR.joblib', 'LSTM.h5', 'RF.joblib', 'SVM.joblib', 'XGB.joblib']


  0%|          | 0/126 [00:00<?, ?it/s]e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [2.0672667e-14 8.3827511e-20 8.3827511e-20 ... 1.0000000e+00 8.8283563e-01
 9.9962938e-01]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
  1%|          | 1/126 [00:13<27:58, 13.43s/it]e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


prediction_prob [2.0672667e-14 8.3827511e-20 8.3827511e-20 ... 1.0000000e+00 8.8283563e-01
 9.9962938e-01]


e:\anaconda\envs\Adversarial_attack\lib\site-packages\tensorflow\python\keras\engine\sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
  1%|          | 1/126 [00:23<49:22, 23.70s/it]


KeyboardInterrupt: 